In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [4]:
# Load the dataset 
df = pd.read_csv("/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv") 

# Check dataset structure
print(df.head())

   Unnamed: 0 label                                               text  \
0         605   ham  Subject: enron methanol ; meter # : 988291\r\n...   
1        2349   ham  Subject: hpl nom for january 9 , 2001\r\n( see...   
2        3624   ham  Subject: neon retreat\r\nho ho ho , we ' re ar...   
3        4685  spam  Subject: photoshop , windows , office . cheap ...   
4        2030   ham  Subject: re : indian springs\r\nthis deal is t...   

   label_num  
0          0  
1          0  
2          0  
3          1  
4          0  


In [5]:
# Drop unnecessary columns (Unnamed: 0)
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

# Check dataset info
print(df.info())

# Ensure correct column names
df = df.rename(columns={"label": "category", "text": "message", "label_num": "label"})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB
None


In [7]:
import re
import string
import nltk
from nltk.corpus import stopwords

stopwords_list = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself",
    "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which",
    "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be",
    "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by",
    "for", "with", "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over",
    "under", "again", "further", "then", "once", "here", "there", "when", "where", "why",
    "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such",
    "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t",
    "can", "will", "just", "don", "should", "now"
])

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [word for word in words if word not in stopwords_list]  # Use manually defined stopwords
    return " ".join(words)


# Apply preprocessing to all messages
df["clean_message"] = df["message"].apply(preprocess_text)

# Display cleaned text
print(df[["message", "clean_message"]].head())


                                             message  \
0  Subject: enron methanol ; meter # : 988291\r\n...   
1  Subject: hpl nom for january 9 , 2001\r\n( see...   
2  Subject: neon retreat\r\nho ho ho , we ' re ar...   
3  Subject: photoshop , windows , office . cheap ...   
4  Subject: re : indian springs\r\nthis deal is t...   

                                       clean_message  
0  subject enron methanol meter follow note gave ...  
1  subject hpl nom january see attached file hpln...  
2  subject neon retreat ho ho ho re around wonder...  
3  subject photoshop windows office cheap main tr...  
4  subject re indian springs deal book teco pvr r...  


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to numerical features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean_message"])
y = df["label"]  # Labels (0 = ham, 1 = spam)

# Display shape of transformed data
print("Shape of X:", X.shape)


Shape of X: (5171, 45658)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# Split dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Naïve Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [11]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test)

# Accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.92
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       742
           1       1.00      0.73      0.84       293

    accuracy                           0.92      1035
   macro avg       0.95      0.86      0.90      1035
weighted avg       0.93      0.92      0.92      1035



In [13]:
# Function to classify new email
def classify_email(email_text):
    processed_email = preprocess_text(email_text)  # Clean text
    vectorized_email = vectorizer.transform([processed_email])  # Convert to numerical features
    prediction = model.predict(vectorized_email)  # Predict
    return "Spam" if prediction[0] == 1 else "Ham"

# Test example
new_email = "Congratulations! you've selected for next round of hackathon."
print("New Email:", classify_email(new_email))


New Email: Ham


In [18]:
# Test example
new_email = "Hurray!! your Machile Learning Course is free on Udemy."
print("New Email:", classify_email(new_email))

New Email: Ham


In [17]:
# Test example
new_email = "Congratulation!! you've won lottery worth $1000."
print("New Email:", classify_email(new_email))

New Email: Spam
